## Autoencoder(Combined with CNN?)

In [1]:
import manga109api
from tensorflow.keras import layers
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import random
import skimage
%matplotlib inline

In [2]:
manga109_root = "../datasets/Manga109/Manga109_released_2021_12_30"
dataset = manga109api.Parser(manga109_root)

In [3]:
def data(is_train = True):
    """define a generator as input to training to not overload the RAM"""
    # set seed so that the size of training/testing set doesn't vary
    random.seed(175069818)
    for book in dataset.books:
        for page in dataset.get_annotation(book)["page"]:
            num = random.randint(0, 19)
            if num % 20 == 0 and is_train: continue
            if num % 20 != 0 and (not is_train): continue
            image = tf.convert_to_tensor((skimage.io.imread("../datasets/Manga109/Manga109_released_2021_12_30/scaled_text_stripped_images/" + book + "/%03d.jpg" % (page["@index"])))/255)
            yield image, image

In [4]:
"""covert generator to tensorflow dataset"""
train_data = tf.data.Dataset.from_generator(data,
                               output_signature=(
                                   tf.TensorSpec(shape=(512,512,3)),
                                   tf.TensorSpec(shape=(512,512,3))
                               )).batch(batch_size=20)
test_data = tf.data.Dataset.from_generator(lambda: data(False),
                               output_signature=(
                                   tf.TensorSpec(shape=(512,512,3)),
                                   tf.TensorSpec(shape=(512,512,3))
                               )).batch(batch_size=20)

In [5]:
"""create autoencoder neural network"""
class Autoencoder(tf.keras.Model):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = tf.keras.Sequential()
        self.decoder = tf.keras.Sequential()
        for i in range(3):
            self.encoder.add(layers.Conv2D(32, (3, 3), activation="relu", padding="same"))
            self.encoder.add(layers.MaxPooling2D((2, 2), padding="same"))
            self.decoder.add(layers.Conv2DTranspose(32, (3, 3), strides=2, activation="relu", padding="same"))
        self.decoder.add(layers.Conv2D(3, (3, 3), activation="sigmoid", padding="same"))
    def call(self, _input):
        encoded = self.encoder(_input)
        decoded = self.decoder(encoded)
        return decoded

autoencoder = Autoencoder()
autoencoder.compile(optimizer="adam", loss="mean_squared_error")

In [6]:
# train the autoencoder
autoencoder.fit(
    x=train_data,
    epochs=2,
    validation_data=test_data
)

Epoch 1/2
    122/Unknown - 141s 1s/step - loss: 0.1274

ResourceExhaustedError: Graph execution error:

Detected at node 'gradient_tape/autoencoder/sequential_1/conv2d_transpose_2/conv2d_transpose/Conv2D' defined at (most recent call last):
    File "C:\Users\olive\anaconda3\envs\Project-Fugu-Manga-Translator\lib\runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\olive\anaconda3\envs\Project-Fugu-Manga-Translator\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\olive\anaconda3\envs\Project-Fugu-Manga-Translator\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "C:\Users\olive\anaconda3\envs\Project-Fugu-Manga-Translator\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\Users\olive\anaconda3\envs\Project-Fugu-Manga-Translator\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "C:\Users

In [ ]:
# book = "ARMS"
# page = {}
# page["@index"] = 4
# skimage.io.imshow(autoencoder.predict((skimage.io.imread("../datasets/Manga109/Manga109_released_2021_12_30/padded_text_stripped_images/" + book + "/%03d.jpg" % (page["@index"]))/255)[np.newaxis])[0])

In [ ]:
# autoencoder.save("model/autoencoder")

In [ ]:
# _autoencoder = tf.keras.models.load_model("model/autoencoder", custom_objects = {"Autoencoder": Autoencoder})

In [ ]:
# _autoencoder.decoder.summary()

In [ ]:
# plt.imshow(skimage.io.imread("../datasets/Manga109/Manga109_released_2021_12_30/padded_text_stripped_images/" + book + "/%03d.jpg" % (page["@index"]))/255)

## References

[https://keras.io/examples/vision/autoencoder/](https://keras.io/examples/vision/autoencoder/)